<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

data.head()

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,413 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121658 files and dire

In [4]:
from datetime import datetime

# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

data['연도'] = data['일시'].dt.year
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 연, 월 별 중앙값
monthly_medians = data.groupby('월').median()
yearly_medians = data.groupby('연도').median()
# 연, 월 별 평균값
monthly_means = data.groupby('월').mean()
yearly_means = data.groupby('연도').mean()

<ipython-input-4-e2deca90dd51>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('월').median()
<ipython-input-4-e2deca90dd51>:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly_medians = data.groupby('연도').median()
<ipython-input-4-e2deca90dd51>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_means = data.groupby('월').mean()
<ipython-input-4-e2deca90dd51>:15: FutureWa

In [5]:
current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22992 non-null  float64       
 2   최저기온    22992 non-null  float64       
 3   일교차     22991 non-null  float64       
 4   강수량     9142 non-null   float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22991 non-null  float64       
 7   일조합     22877 non-null  float64       
 8   일사합     18137 non-null  float64       
 9   일조율     22630 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  연도      22995 non-null  int64         
 12  월       22995 non-null  int64         
 13  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(3)
memory usage: 2.6 MB
None


In [6]:
# 결측값 대체
data.fillna(monthly_medians, inplace=True)

for column in data.columns:
    if data[column].isnull().any():  # 결측값이 있는 경우에만 처리
        data[column+'_중앙값'] = data.apply(lambda row: monthly_medians.loc[row['월'], column], axis=1)
        data[column].fillna(data[column+'_중앙값'], inplace=True)
        data.drop(column+'_중앙값', axis=1, inplace=True)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22995 non-null  float64       
 2   최저기온    22995 non-null  float64       
 3   일교차     22995 non-null  float64       
 4   강수량     22995 non-null  float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22995 non-null  float64       
 7   일조합     22995 non-null  float64       
 8   일사합     22995 non-null  float64       
 9   일조율     22995 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  연도      22995 non-null  int64         
 12  월       22995 non-null  int64         
 13  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(3)
memory usage: 3.1 MB
None


In [7]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

import warnings
warnings.filterwarnings(action='ignore')

In [9]:
train_input = train[:16425]
test_input = train[:6570]

train_target = target[:16425]
test_target = target[:6570]

In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_input, train_target)

rf.score(train_input, train_target)

0.9996613064710366

In [12]:
rf.score(test_input, test_target)

0.9996454430482914

In [13]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_scaled, train_target)

RandomForestRegressor()

In [15]:
rf_pred = rf.predict(test_scaled)

from sklearn.metrics import mean_absolute_error
rf_mae = mean_absolute_error(rf_pred, test_target)

rf_mae

0.1484972602739723

In [16]:
from sklearn.model_selection import GridSearchCV

# 탐색할 하이퍼파라미터 값 지정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_scaled, train_target)

# 최적의 하이퍼파라미터 출력
print("Best Parameters: ", grid_search.best_params_)

KeyboardInterrupt: ignored

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 랜덤포레스트 모델 생성
rf = RandomForestRegressor()

# 탐색할 하이퍼파라미터 값 지정
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV를 사용하여 랜덤하게 하이퍼파라미터 탐색
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(train_scaled, train_target)

# 최적의 하이퍼파라미터 출력
print("Best Parameters: ", random_search.best_params_)


KeyboardInterrupt: ignored

In [19]:
pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.0 MB/s eta 0:00:00


In [21]:
from skopt import BayesSearchCV

# 탐색할 하이퍼파라미터 값 지정 (None 값을 다른 값으로 대체)
param_space = {
    'n_estimators': (50, 200),
    'max_depth': (1, 30),  # None을 적절한 값으로 대체
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
}

# Bayesian Optimization을 사용하여 하이퍼파라미터 탐색
opt = BayesSearchCV(rf, param_space, n_iter=10, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
opt.fit(train_scaled, train_target)

# 최적의 하이퍼파라미터 출력
print("Best Parameters: ", opt.best_params_)

Best Parameters:  OrderedDict([('max_depth', 25), ('min_samples_leaf', 4), ('min_samples_split', 4), ('n_estimators', 193)])


In [26]:
rf = RandomForestRegressor(max_depth = 25, min_samples_leaf = 4, min_samples_split = 4, n_estimators = 193)

rf.fit(train_scaled, train_target)

RandomForestRegressor(max_depth=25, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=193)

In [27]:
rf_train_pred = rf.predict(train_scaled)
rf_test_pred = rf.predict(test_scaled)

rf_train_mae = mean_absolute_error(rf_train_pred, train_target)
rf_test_mae = mean_absolute_error(rf_test_pred, test_target)

print(rf_train_mae)
print(rf_test_mae)

0.23415411409430256
0.24533809057678124


In [29]:
submission['평균기온'] = rf_test_pred[:358]
print(submission.head(30))

            일시       평균기온
0   2023-01-01  -1.687236
1   2023-01-02  -2.181296
2   2023-01-03   3.464391
3   2023-01-04   6.427569
4   2023-01-05  -4.080143
5   2023-01-06  -5.424372
6   2023-01-07   1.178173
7   2023-01-08   1.848372
8   2023-01-09   2.260578
9   2023-01-10   1.923334
10  2023-01-11  -1.784666
11  2023-01-12  -3.663829
12  2023-01-13  -3.662116
13  2023-01-14  -2.036136
14  2023-01-15  -5.268023
15  2023-01-16  -4.923871
16  2023-01-17  -5.664201
17  2023-01-18  -2.974505
18  2023-01-19   2.126944
19  2023-01-20  -0.799986
20  2023-01-21  -6.723405
21  2023-01-22  -9.339594
22  2023-01-23 -13.081159
23  2023-01-24 -12.679708
24  2023-01-25  -7.590178
25  2023-01-26  -7.875513
26  2023-01-27 -10.908784
27  2023-01-28 -11.285364
28  2023-01-29  -8.060372
29  2023-01-30  -0.699935


In [30]:
submission.to_csv("DEAGYUL_15_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -1.687236
1  2023-01-02 -2.181296
2  2023-01-03  3.464391
3  2023-01-04  6.427569
4  2023-01-05 -4.080143


스케일링을 먹이면 점수가 전부 안좋게 나와

스케일링이 문제인가 아니면 세트 분리해서 학습한게 문제일까

학습할때마다 오래걸려서 파일 분리하기도 그런데 개빡친다